In [15]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_parquet('./data/data_df_hcm.parquet')
target_df = pd.read_parquet('./data/target_df_hcm.parquet')


In [17]:
df

,nearest_5_district,nearest_8_street,nearest_2_ward,nearest_4_district,nearest_1_district,typeOfRealEstate,houseDirection,nearest_5_street,nearest_7_street,street,...,distance_hn_CongtruongLamSon,distance_hn_DuongNguyenHuequan1TPHCM,nearest_4_lat,num_of_atm_in_2000m_radius,num_of_place_of_worship_in_2000m_radius,num_of_restaurant_in_1000m_radius,numberOfBathRooms,distance_hn_DuongNamKyKhoiNghia,event_timestamp,realestate_id
24794,11,1730,13,11,11,3,0,857,756,253,...,7.647704,7.771465,10.790462,106,98,97,4.478184,7.835349,2023-04-01,24794
37176,10,2986,547,10,10,3,0,4053,3429,493,...,8.551525,8.572820,10.786360,35,72,15,4.478184,8.578963,2023-04-03,37176
6905,11,100,518,11,11,3,0,495,100,2252,...,7.143411,6.899570,10.766617,129,109,303,4.478184,6.688504,2023-05-18,6905
6412,10,3640,438,14,14,0,1,3062,1479,1551,...,8.428535,8.488024,10.803249,58,48,13,2.000000,8.519420,2023-07-13,6412
24436,18,1998,28,18,18,1,4,208,2642,219,...,8.776880,8.799135,10.799805,7,9,9,3.000000,8.820474,2023-07-14,24436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2697,1,357,116,1,1,1,0,3299,506,153,...,8.728899,8.680754,10.736959,9,2,3,2.000000,8.654018,2024-02-16,2697
22001,20,963,30,14,20,3,0,91,311,30,...,8.216211,8.298848,10.805246,57,55,15,5.000000,8.345146,2024-02-16,22001
21481,13,2268,399,13,13,1,7,2372,1469,2733,...,9.479880,9.503459,10.886026,3,1,0,1.000000,9.516926,2024-02-16,21481
10914,10,599,140,30,10,3,0,3708,3451,1047,...,8.967848,8.989615,10.805946,17,12,3,4.478184,8.998477,2024-02-16,10914


In [18]:
target_df

,target,event_timestamp,realestate_id
24794,450.000000,2023-04-01,24794
37176,176.000000,2023-04-03,37176
6905,406.250000,2023-05-18,6905
6412,69.863014,2023-07-13,6412
24436,185.000000,2023-07-14,24436
...,...,...,...
2697,96.250000,2024-02-16,2697
22001,123.595506,2024-02-16,22001
21481,51.875000,2024-02-16,21481
10914,257.352941,2024-02-16,10914


In [19]:
import json

FS = json.load(open('./data/full_version.json', 'r'))
cat_cols = FS['cat_cols']
num_cols = FS['num_cols']
all_cols = cat_cols + num_cols

df = df[all_cols]

In [20]:
categorical_features_indices = [i for i, c in enumerate(all_cols) if c in cat_cols]

In [21]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
import random

def create_model(cat_idxs = None):
    if cat_idxs:
        model = CatBoostRegressor(cat_features=cat_idxs, iterations=5000, verbose = 100, random_state=random.randint(3, 1000), task_type='GPU', devices='0:1')
    else:
        model = CatBoostRegressor(iterations=5000, verbose = 100, random_state=random.randint(3, 1000), task_type='GPU', devices='0:1')
    search_params = {'learning_rate': [0.01, 0.08, 0.15]}
    clf = GridSearchCV(model, search_params, scoring=['explained_variance', 'max_error', 'neg_root_mean_squared_error', 'r2', 'neg_median_absolute_error', 'neg_mean_absolute_percentage_error'], refit='neg_root_mean_squared_error', cv=5)
    return clf

In [22]:
model = create_model(categorical_features_indices)

In [23]:
full_df = pd.concat([df, target_df['target']], axis = 1)

In [24]:
full_df.shape

(44209, 184)

In [25]:
train_df = full_df.iloc[:-35000]
test_df = full_df.iloc[-35000:]

In [26]:
def train_test_split_by_col(train_df, test_df, X_cols, y_col):
    X_train, X_test, y_train, y_test = train_df[X_cols], test_df[X_cols], train_df[y_col], test_df[y_col]

    return X_train, X_test, np.log(y_train), np.log(y_test)


In [27]:
target_feature = 'target'

In [28]:
X_train, X_test, y_train, y_test = train_test_split_by_col(train_df = train_df, test_df = test_df, X_cols = all_cols, y_col = target_feature)
model.fit(X_train, y_train)

0:	learn: 0.7982113	total: 98.4ms	remaining: 8m 11s
100:	learn: 0.4901289	total: 7.27s	remaining: 5m 52s
200:	learn: 0.4058764	total: 13.5s	remaining: 5m 23s
300:	learn: 0.3801148	total: 19.6s	remaining: 5m 6s
400:	learn: 0.3692692	total: 26.2s	remaining: 5m
500:	learn: 0.3624457	total: 32.4s	remaining: 4m 51s
600:	learn: 0.3573723	total: 39.1s	remaining: 4m 46s
700:	learn: 0.3530691	total: 45.3s	remaining: 4m 37s
800:	learn: 0.3494640	total: 52.4s	remaining: 4m 34s
900:	learn: 0.3463947	total: 58.8s	remaining: 4m 27s
1000:	learn: 0.3431714	total: 1m 5s	remaining: 4m 22s
1100:	learn: 0.3403305	total: 1m 12s	remaining: 4m 16s
1200:	learn: 0.3380016	total: 1m 19s	remaining: 4m 12s
1300:	learn: 0.3357316	total: 1m 26s	remaining: 4m 5s
1400:	learn: 0.3337899	total: 1m 33s	remaining: 4m
1500:	learn: 0.3319369	total: 1m 40s	remaining: 3m 53s
1600:	learn: 0.3300733	total: 1m 46s	remaining: 3m 46s
1700:	learn: 0.3284301	total: 1m 52s	remaining: 3m 38s
1800:	learn: 0.3266077	total: 1m 58s	remai

0:	learn: 0.7983337	total: 65.9ms	remaining: 5m 29s
100:	learn: 0.4929009	total: 6.49s	remaining: 5m 15s
200:	learn: 0.4081323	total: 12.5s	remaining: 4m 58s
300:	learn: 0.3822928	total: 19.4s	remaining: 5m 3s
400:	learn: 0.3715444	total: 26.1s	remaining: 4m 59s
500:	learn: 0.3643468	total: 32.7s	remaining: 4m 53s
600:	learn: 0.3590594	total: 38.9s	remaining: 4m 44s
700:	learn: 0.3550024	total: 45.6s	remaining: 4m 39s
800:	learn: 0.3512305	total: 52.2s	remaining: 4m 33s
900:	learn: 0.3480577	total: 58.6s	remaining: 4m 26s
1000:	learn: 0.3449483	total: 1m 5s	remaining: 4m 21s
1100:	learn: 0.3422260	total: 1m 11s	remaining: 4m 14s
1200:	learn: 0.3397410	total: 1m 18s	remaining: 4m 8s
1300:	learn: 0.3373618	total: 1m 24s	remaining: 4m 1s
1400:	learn: 0.3351982	total: 1m 31s	remaining: 3m 55s
1500:	learn: 0.3330962	total: 1m 39s	remaining: 3m 51s
1600:	learn: 0.3313618	total: 1m 45s	remaining: 3m 44s
1700:	learn: 0.3297678	total: 1m 54s	remaining: 3m 41s
1800:	learn: 0.3282593	total: 2m 1s

0:	learn: 0.8059249	total: 61.6ms	remaining: 5m 8s
100:	learn: 0.4952012	total: 6.18s	remaining: 4m 59s
200:	learn: 0.4090816	total: 13s	remaining: 5m 10s
300:	learn: 0.3829450	total: 19.9s	remaining: 5m 10s
400:	learn: 0.3718552	total: 26.4s	remaining: 5m 2s
500:	learn: 0.3649136	total: 33.9s	remaining: 5m 4s
600:	learn: 0.3597744	total: 40.2s	remaining: 4m 54s
700:	learn: 0.3559016	total: 47.3s	remaining: 4m 49s
800:	learn: 0.3526075	total: 54s	remaining: 4m 43s
900:	learn: 0.3491089	total: 1m	remaining: 4m 33s
1000:	learn: 0.3461060	total: 1m 6s	remaining: 4m 27s
1100:	learn: 0.3434845	total: 1m 13s	remaining: 4m 21s
1200:	learn: 0.3409287	total: 1m 20s	remaining: 4m 14s
1300:	learn: 0.3386179	total: 1m 27s	remaining: 4m 7s
1400:	learn: 0.3366674	total: 1m 34s	remaining: 4m 2s
1500:	learn: 0.3347975	total: 1m 41s	remaining: 3m 55s
1600:	learn: 0.3331595	total: 1m 47s	remaining: 3m 48s
1700:	learn: 0.3316834	total: 1m 54s	remaining: 3m 41s
1800:	learn: 0.3302989	total: 2m 1s	remainin

0:	learn: 0.7961460	total: 37.3ms	remaining: 3m 6s
100:	learn: 0.4884038	total: 6.71s	remaining: 5m 25s
200:	learn: 0.4028706	total: 13.6s	remaining: 5m 24s
300:	learn: 0.3767700	total: 20.6s	remaining: 5m 21s
400:	learn: 0.3657589	total: 26.4s	remaining: 5m 2s
500:	learn: 0.3590278	total: 33.4s	remaining: 4m 59s
600:	learn: 0.3541599	total: 40.5s	remaining: 4m 56s
700:	learn: 0.3502642	total: 46.5s	remaining: 4m 44s
800:	learn: 0.3466758	total: 52.7s	remaining: 4m 36s
900:	learn: 0.3436366	total: 59.3s	remaining: 4m 29s
1000:	learn: 0.3407081	total: 1m 5s	remaining: 4m 22s
1100:	learn: 0.3377726	total: 1m 12s	remaining: 4m 15s
1200:	learn: 0.3352361	total: 1m 18s	remaining: 4m 7s
1300:	learn: 0.3328624	total: 1m 24s	remaining: 4m 1s
1400:	learn: 0.3307710	total: 1m 31s	remaining: 3m 55s
1500:	learn: 0.3290729	total: 1m 38s	remaining: 3m 50s
1600:	learn: 0.3275182	total: 1m 46s	remaining: 3m 45s
1700:	learn: 0.3259530	total: 1m 52s	remaining: 3m 38s
1800:	learn: 0.3245365	total: 1m 59s

0:	learn: 0.7990969	total: 35.7ms	remaining: 2m 58s
100:	learn: 0.4862133	total: 6.17s	remaining: 4m 59s
200:	learn: 0.4000270	total: 13.2s	remaining: 5m 15s
300:	learn: 0.3747153	total: 20.1s	remaining: 5m 13s
400:	learn: 0.3639089	total: 26.5s	remaining: 5m 4s
500:	learn: 0.3573283	total: 33.2s	remaining: 4m 58s
600:	learn: 0.3524098	total: 40.3s	remaining: 4m 55s
700:	learn: 0.3484293	total: 47.6s	remaining: 4m 51s
800:	learn: 0.3451199	total: 53.7s	remaining: 4m 41s
900:	learn: 0.3416208	total: 1m	remaining: 4m 35s
1000:	learn: 0.3389063	total: 1m 7s	remaining: 4m 29s
1100:	learn: 0.3359950	total: 1m 14s	remaining: 4m 24s
1200:	learn: 0.3331258	total: 1m 22s	remaining: 4m 19s
1300:	learn: 0.3307887	total: 1m 29s	remaining: 4m 13s
1400:	learn: 0.3288916	total: 1m 37s	remaining: 4m 9s
1500:	learn: 0.3269437	total: 1m 44s	remaining: 4m 2s
1600:	learn: 0.3249892	total: 1m 51s	remaining: 3m 56s
1700:	learn: 0.3234185	total: 1m 58s	remaining: 3m 49s
1800:	learn: 0.3220104	total: 2m 5s	re

...

0:	learn: 0.7693635	total: 35.5ms	remaining: 2m 57s
100:	learn: 0.3528433	total: 6.2s	remaining: 5m
200:	learn: 0.3357951	total: 12.6s	remaining: 5m 1s
300:	learn: 0.3253319	total: 18.5s	remaining: 4m 48s
400:	learn: 0.3191832	total: 24.9s	remaining: 4m 45s
500:	learn: 0.3138084	total: 31.5s	remaining: 4m 43s
600:	learn: 0.3091704	total: 37.5s	remaining: 4m 34s
700:	learn: 0.3057679	total: 43.5s	remaining: 4m 26s
800:	learn: 0.3028774	total: 50s	remaining: 4m 22s
900:	learn: 0.2991771	total: 56.2s	remaining: 4m 15s
1000:	learn: 0.2965903	total: 1m 2s	remaining: 4m 10s
1100:	learn: 0.2946720	total: 1m 9s	remaining: 4m 6s
1200:	learn: 0.2927977	total: 1m 15s	remaining: 4m
1300:	learn: 0.2905858	total: 1m 21s	remaining: 3m 52s
1400:	learn: 0.2885523	total: 1m 28s	remaining: 3m 46s
1500:	learn: 0.2869273	total: 1m 34s	remaining: 3m 40s
1600:	learn: 0.2856021	total: 1m 41s	remaining: 3m 35s
1700:	learn: 0.2842881	total: 1m 48s	remaining: 3m 29s
1800:	learn: 0.2828251	total: 1m 54s	remaining

0:	learn: 0.7584997	total: 71ms	remaining: 5m 54s
100:	learn: 0.3463089	total: 6.31s	remaining: 5m 6s
200:	learn: 0.3283879	total: 12.7s	remaining: 5m 2s
300:	learn: 0.3181816	total: 19.6s	remaining: 5m 5s
400:	learn: 0.3106387	total: 26.1s	remaining: 4m 59s
500:	learn: 0.3049343	total: 32.9s	remaining: 4m 55s
600:	learn: 0.2998586	total: 39.6s	remaining: 4m 49s
700:	learn: 0.2961661	total: 46s	remaining: 4m 42s
800:	learn: 0.2932536	total: 52.7s	remaining: 4m 36s
900:	learn: 0.2908986	total: 59.3s	remaining: 4m 29s
1000:	learn: 0.2887538	total: 1m 6s	remaining: 4m 24s
1100:	learn: 0.2860359	total: 1m 12s	remaining: 4m 17s
1200:	learn: 0.2838787	total: 1m 19s	remaining: 4m 9s
1300:	learn: 0.2813876	total: 1m 25s	remaining: 4m 2s
1400:	learn: 0.2800511	total: 1m 32s	remaining: 3m 57s
1500:	learn: 0.2780366	total: 1m 38s	remaining: 3m 49s
1600:	learn: 0.2764616	total: 1m 44s	remaining: 3m 42s
1700:	learn: 0.2747925	total: 1m 51s	remaining: 3m 36s
1800:	learn: 0.2732657	total: 1m 57s	rema

0:	learn: 0.7612363	total: 73ms	remaining: 6m 5s
100:	learn: 0.3448367	total: 6.42s	remaining: 5m 11s
200:	learn: 0.3265853	total: 13s	remaining: 5m 9s
300:	learn: 0.3157832	total: 19.9s	remaining: 5m 10s
400:	learn: 0.3088549	total: 27.1s	remaining: 5m 10s
500:	learn: 0.3035289	total: 33.5s	remaining: 5m
600:	learn: 0.2988660	total: 40.6s	remaining: 4m 57s
700:	learn: 0.2951720	total: 47.2s	remaining: 4m 49s
800:	learn: 0.2927028	total: 54.5s	remaining: 4m 45s
900:	learn: 0.2903053	total: 1m	remaining: 4m 37s
1000:	learn: 0.2879850	total: 1m 7s	remaining: 4m 29s
1100:	learn: 0.2858455	total: 1m 13s	remaining: 4m 21s
1200:	learn: 0.2838324	total: 1m 20s	remaining: 4m 13s
1300:	learn: 0.2815644	total: 1m 26s	remaining: 4m 6s
1400:	learn: 0.2800318	total: 1m 33s	remaining: 3m 59s
1500:	learn: 0.2783155	total: 1m 40s	remaining: 3m 53s
1600:	learn: 0.2766246	total: 1m 46s	remaining: 3m 46s
1700:	learn: 0.2753598	total: 1m 52s	remaining: 3m 38s
1800:	learn: 0.2739814	total: 1m 59s	remaining

0:	learn: 0.7268589	total: 34ms	remaining: 2m 50s
100:	learn: 0.3349431	total: 6.67s	remaining: 5m 23s
200:	learn: 0.3158141	total: 12.9s	remaining: 5m 9s
300:	learn: 0.3050316	total: 20s	remaining: 5m 11s
400:	learn: 0.2999763	total: 27.1s	remaining: 5m 10s
500:	learn: 0.2945220	total: 34.2s	remaining: 5m 6s
600:	learn: 0.2882320	total: 41.5s	remaining: 5m 3s
700:	learn: 0.2852923	total: 47.8s	remaining: 4m 52s
800:	learn: 0.2818266	total: 55.1s	remaining: 4m 49s
900:	learn: 0.2792641	total: 1m 2s	remaining: 4m 43s
1000:	learn: 0.2770872	total: 1m 9s	remaining: 4m 36s
1100:	learn: 0.2752320	total: 1m 15s	remaining: 4m 26s
1200:	learn: 0.2732795	total: 1m 22s	remaining: 4m 19s
1300:	learn: 0.2711628	total: 1m 28s	remaining: 4m 12s
1400:	learn: 0.2689686	total: 1m 35s	remaining: 4m 4s
1500:	learn: 0.2673761	total: 1m 42s	remaining: 3m 58s
1600:	learn: 0.2657722	total: 1m 49s	remaining: 3m 51s
1700:	learn: 0.2640287	total: 1m 56s	remaining: 3m 45s
1800:	learn: 0.2625779	total: 2m 3s	rema

0:	learn: 0.7239793	total: 41.7ms	remaining: 3m 28s
100:	learn: 0.3370021	total: 6.71s	remaining: 5m 25s
200:	learn: 0.3171306	total: 12.6s	remaining: 4m 59s
300:	learn: 0.3061331	total: 18.8s	remaining: 4m 53s
400:	learn: 0.3002740	total: 25.5s	remaining: 4m 52s
500:	learn: 0.2956165	total: 32s	remaining: 4m 47s
600:	learn: 0.2895590	total: 38.3s	remaining: 4m 40s
700:	learn: 0.2862162	total: 44.4s	remaining: 4m 32s
800:	learn: 0.2833139	total: 51.4s	remaining: 4m 29s
900:	learn: 0.2805745	total: 58.6s	remaining: 4m 26s
1000:	learn: 0.2783387	total: 1m 4s	remaining: 4m 18s
1100:	learn: 0.2756405	total: 1m 11s	remaining: 4m 13s
1200:	learn: 0.2726890	total: 1m 17s	remaining: 4m 6s
1300:	learn: 0.2701262	total: 1m 24s	remaining: 4m
1400:	learn: 0.2675576	total: 1m 31s	remaining: 3m 54s
1500:	learn: 0.2655229	total: 1m 38s	remaining: 3m 50s
1600:	learn: 0.2633819	total: 1m 46s	remaining: 3m 45s
1700:	learn: 0.2621446	total: 1m 52s	remaining: 3m 38s
1800:	learn: 0.2604286	total: 1m 58s	re

In [ ]:
model.predict(X_test)[:10]

In [ ]:
from joblib import dump, load

In [ ]:
estimator = model.best_estimator_
dump(estimator, "./model/hcm/cat/v2/model.joblib")

In [ ]:
load_model = load("./model/hcm/cat/v2/model.joblib")
load_model.predict(X_test)[:10]